In [1]:
#r "D:\TFS_Sources\Github\Compilation\trmcnealy\DataStorage\bin\Debug\netcoreapp3.1\Kokkos.NET.dll"
#r "D:\TFS_Sources\Github\Compilation\trmcnealy\DataStorage\bin\Debug\netcoreapp3.1\PostgreSql.NET.dll"
#r "D:\TFS_Sources\Github\Compilation\trmcnealy\DataStorage\bin\Debug\netcoreapp3.1\Microsoft.Data.Analysis.dll"
#r "D:\TFS_Sources\Github\Compilation\trmcnealy\DataStorage\bin\Debug\netcoreapp3.1\Engineering.DataSource.dll"
#r "D:\TFS_Sources\Github\Compilation\trmcnealy\OilGas.Data\bin\Debug\netcoreapp3.1\OilGas.Data.dll"
#r "D:\TFS_Sources\Github\Compilation\trmcnealy\OilGas.Data\bin\Debug\netcoreapp3.1\OilGas.Data.RRC.Texas.dll"

The below script needs to be able to find the current output cell; this is an easy method to get it.

In [2]:
using Microsoft.Data.Analysis;
using Engineering.DataSource;
using Engineering.DataSource.GeoSpatial;

In [3]:
var dbConnection = new DbConnector("timothyrmcnealy.com", 5432).Connect("db_user", "dbAccess", "OilGas").Result;

user db_user trying to connect to OilGas @ timothyrmcnealy.com:5432 with dbAccess
Connecting....user db_user connected to OilGas with dbAccess


In [4]:
var mapviewExtents = dbConnection.Query("SELECT min(\"SurfaceEasting83\") AS \"MinEasting\", max(\"SurfaceEasting83\") AS \"MaxEasting\", min(\"SurfaceNorthing83\") AS \"MinNorthing\", max(\"SurfaceNorthing83\") AS \"MaxNorthing\"FROM \"ShapeFileLocation\";").Result;

In [5]:
var mapviewExtentsDF = mapviewExtents.ToDataFrame();

In [6]:
var mapview_extents = mapviewExtentsDF.ToJson();

In [7]:
mapview_extents

[
  {
    "MinEasting": -11088374.904767515,
    "MaxEasting": -10714043.420104431,
    "MinNorthing": 3250415.806441195,
    "MaxNorthing": 3539269.13245472
  }
]

In [8]:
var wellLocations = dbConnection.Query("SELECT "+
"    Wells.\"Api\",     "+
"    ReservoirDatas.\"ReservoirName\", "+
"    ReservoirDatas.\"ReservoirDepth\",     "+
"    GasPropertiess.\"SpecificGravity\" AS \"GasSpecificGravity\", "+
"    OilPropertiess.\"Density\" AS \"OilApiGravity\", "+
"    \"ShapeFileLocation\".\"SurfaceEasting83\" AS \"SurfaceEasting\", "+
"    \"ShapeFileLocation\".\"SurfaceNorthing83\" AS \"SurfaceNorthing\", "+
"    \"ShapeFileLocation\".\"BottomEasting83\" AS \"BottomEasting\", "+
"    \"ShapeFileLocation\".\"BottomNorthing83\" AS \"BottomNorthing\" "+
"FROM ( "+
"    SELECT \"Id\", \"Api\" "+
"    FROM \"Well\" "+
") Wells "+
"LEFT JOIN ( "+
"    SELECT \"Id\", \"ReservoirName\", \"ReservoirDepth\", \"WellId\" "+
"    FROM \"ReservoirData\" "+
"    WHERE \"ReservoirData\".\"ReservoirName\" LIKE 'EAGLE%' "+
") ReservoirDatas "+
"ON Wells.\"Id\"=ReservoirDatas.\"WellId\" "+
"LEFT JOIN ( "+
"    SELECT \"SpecificGravity\", \"ReservoirDataId\" "+
"    FROM \"GasProperties\" "+
") GasPropertiess "+
"ON ReservoirDatas.\"Id\"=GasPropertiess.\"ReservoirDataId\" "+
"LEFT JOIN ( "+
"    SELECT \"Density\", \"ReservoirDataId\" "+
"    FROM \"OilProperties\" "+
") OilPropertiess "+
"ON ReservoirDatas.\"Id\"=OilPropertiess.\"ReservoirDataId\" "+
"LEFT JOIN \"ShapeFileLocation\" "+
"ON Wells.\"Api\"=\"ShapeFileLocation\".\"Api\" "+
"WHERE \"ShapeFileLocation\".\"SurfaceEasting83\" IS NOT NULL AND GasPropertiess.\"SpecificGravity\" IS NOT NULL AND OilPropertiess.\"Density\" IS NOT NULL").Result;

In [9]:
DataFrame wellLocationsDf = wellLocations.ToDataFrame();
var wellLocationsGeoJson = wellLocationsDf.ToGeoJson();

In [10]:
#!js
notebookScope.plot = (htmlElement, variableName) => {

    require([
        "esri/Map",
        "esri/views/MapView",
        "esri/layers/Layer",
        "esri/layers/FeatureLayer",
        "esri/layers/GeoJSONLayer",
        "esri/layers/support/Field",
        "esri/widgets/Legend",
        "esri/widgets/Expand",
        "esri/renderers/smartMapping/creators/color",
        "esri/renderers/smartMapping/symbology/color",
        "esri/renderers/smartMapping/statistics/histogram",
        "esri/widgets/smartMapping/ColorSlider",
        "esri/Color",
        "esri/intl",
        "esri/widgets/Fullscreen",
        "esri/PopupTemplate"
    ], function (Map,
                 MapView,
                 Layer,
                 FeatureLayer,
                 GeoJSONLayer,
                 Field,
                 Legend,
                 Expand,
                 colorRendererCreator,
                 colorSymbology,
                 histogram,
                 ColorSlider,
                 Color,
                 intl,
                 Fullscreen,
                 PopupTemplate) {
        
        //Loader.setDefaultOptions({ url: "https://js.arcgis.com/4.15/" });


        interactive.csharp.getVariable(variableName).then(variableValue => {
            window[variableName] = JSON.parse(variableValue);
        });

        interactive.csharp.getVariable("mapview_extents").then(variableValue => {
            const value = JSON.parse(variableValue);
            window["MinEasting"] = value[0].MinEasting;
            window["MaxEasting"] = value[0].MaxEasting;
            window["MinNorthing"] = value[0].MinNorthing;
            window["MaxNorthing"] = value[0].MaxNorthing;
        });

        let attributeLayerView;

        const map = new Map({
            basemap: "dark-gray"
        });

        //variableNames`{UID}`.forEach(variableName => {

        //    let variable = window[variableName];

        //    var features = new Array(variable.length);

        //    var idCounter = 1;

        //    variable.forEach(variableValue => {

        //        features.push({
        //            geometry: {
        //                type: "point",
        //                x: variableValue.SurfaceEasting,
        //                y: variableValue.SurfaceNorthing
        //            },
        //            attributes: {
        //                ObjectID: idCounter,
        //                Api: variableValue.Api,
        //                GasSpecificGravity: variableValue.GasSpecificGravity,
        //                OilApiGravity: variableValue.OilApiGravity,
        //                ReservoirDepth: variableValue.ReservoirDepth
        //            }

        //        });

        //        idCounter = idCounter + 1;

        //    });

        //    //createFilter("view-`{ID}`", "`{ID}`", ["GasSpecificGravity", "OilApiGravity", "ReservoirDepth"], attributeLayerView);

        //    const popupTemplate = new PopupTemplate({
        //        title: "{Api}",
        //        description: "--description--"
        //    });

        //    const layer = new FeatureLayer({
        //        source: features,
        //        objectIdField: "ObjectID",
        //        infoTemplate: popupTemplate,
        //        outFields: ["ATTRIBUTE"]
        //    });

        //    map.add(layer);

        //});

        const centerX = (window["MaxEasting"] + window["MinEasting"]) / 2.0;
        const centerY = (window["MaxNorthing"] + window["MinNorthing"]) / 2.0;

        const view = new MapView({
            container: "view-`{ID}`",
            map: map,
            zoom: 5,
            center: [centerX, centerY],
            resizeAlign: "top-left"
        });

        //const legend = new Expand({
        //    content: new Legend({
        //        view: view,
        //        style: "card" // other styles include 'classic'
        //    }),
        //    view: view,
        //    expanded: false
        //});

        //view.ui.add(legend, "bottom-left");

        //var fullscreenDiv = document.getElementById("view-`{ID}`");

        //const fullscreen = new Fullscreen({
        //    view: view,
        //    element: fullscreenDiv
        //});

        //view.ui.add(fullscreen, "top-right");

        //const home = new Home({
        //    view: view
        //});

        //view.ui.add(home, "top-left");

        //view.whenLayerView(layer).then(function (layerView) {

        //    attributeLayerView = layerView;

        //    const attributeElement = document.getElementById("view-`{ID}`-filter");

        //    attributeElement.style.visibility = "visible";

        //    const attributeExpand = new Expand({
        //        view: view,
        //        content: attributeElement,
        //        expandIconClass: "esri-icon-filter",
        //        group: "top-left"
        //    });

        //    attributeExpand.watch("expanded",
        //        function () {
        //            if (!attributeExpand.expanded) {
        //                attributeLayerView.filter = null;
        //            }
        //        });

        //    view.ui.add(attributeExpand, "top-left");
        //});

    });



}

In [11]:
#!js
var script = document.createElement("script");
script.setAttribute("src", "https://js.arcgis.com/4.15/");
document.getElementsByTagName("head")[0].appendChild(script);

#!html
<div id="view" width="100%"></div>

#!js
notebookScope.plot("div#view", "wellLocationsGeoJson");

In [ ]:
ArcGIS map = new ArcGIS(1000,750);

In [ ]:
map.CreateLayersFrom(nameof(wellLocationsGeoJson));

In [ ]:
display(map.GetHtmlContent());

In [ ]:
#!javascript
interactive.csharp.getVariable("mapview_extents").then(data => { 
    alert(JSON.parse(data));
});